In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)
SEED = 0
test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)

# for r = r, reduce all the sets to 20% of the original size

r = .5

train_labels = train_labels[:int(r*len(train_labels))]
train_features = train_features[:int(r*len(train_features))]
test_labels = test_labels[:int(r*len(test_labels))]
test_features = test_features[:int(r*len(test_features))]
val_labels = val_labels[:int(r*len(val_labels))]
val_features = val_features[:int(r*len(val_features))]



In [ ]:
from Quantum_temporal_kernel.SVM_models.QuantumSVM_2 import QuantumSVM
quantum_svm = QuantumSVM(n_qubits=2)
quantum_svm.compute_kernels(train_features, test_features)


Generating random coefficients for the Hamiltonian...


100%|██████████| 150/150 [05:01<00:00,  2.01s/it]


K_train_dict generated


100%|██████████| 150/150 [17:19<00:00,  6.93s/it]

K_test_dict generated


In [24]:
from sklearn.svm import SVC
base_learner = SVC(C=100)
from MKLpy.algorithms import EasyMKL, AverageMKL

#KLtr must be a list of kernels from the dictionary kernels_train

#convert the dictionary to a list
KLtr = []
for key in quantum_svm.K_train_dict:
    KLtr.append(quantum_svm.K_train_dict[key])
Ytr = train_labels

KLte = []
for key in quantum_svm.K_test_dict:
    KLte.append(quantum_svm.K_test_dict[key])
Yte = test_labels


In [25]:
mkl = AverageMKL().fit(KLtr, Ytr)       #combine kernels and train the classifier
y_preds  = mkl.predict(KLte)            #predict the output class
y_scores = mkl.decision_function(KLte) 

from sklearn.metrics import accuracy_score, roc_auc_score
accuracy = accuracy_score(Yte, y_preds)
roc_auc = roc_auc_score(Yte, y_scores)
print ('Accuracy score: %.4f, roc AUC score: %.4f' % (accuracy, roc_auc))

Accuracy score: 0.8200, roc AUC score: 0.9040


In [26]:
mkl = EasyMKL().fit(KLtr, Ytr)       #combine kernels and train the classifier
y_preds  = mkl.predict(KLte)            #predict the output class
y_scores = mkl.decision_function(KLte) 

from sklearn.metrics import accuracy_score, roc_auc_score
accuracy = accuracy_score(Yte, y_preds)
roc_auc = roc_auc_score(Yte, y_scores)
print ('Accuracy score: %.4f, roc AUC score: %.4f' % (accuracy, roc_auc))

Accuracy score: 0.7200, roc AUC score: 0.8848


In [35]:
#print solution attributes
solution_obj = mkl.solution

weights = mkl.solution.weights
print(len(weights))

150
